# Persyaratan Teknis:
- Gunakan minimal dua operator pengolahan citra dasar (misalnya,
Gaussian blur, edge detection) untuk meningkatkan kualitas data.
- Implementasikan minimal dua metode detektor atau deskriptor fitur lokal
(misalnya, SIFT, SURF, ORB).
- Buat sistem untuk menemukan korespondensi antara beberapa citra
(misalnya, menggunakan keypoints matching) dan menerapkannya dalam
pengenalan.
- Bangun sistem pengenalan berbasis fitur yang mampu mengklasifikasikan
atau mengidentifikasi objek dalam citra secara otomatis.

In [1]:
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report 
from xgboost import XGBClassifier

In [2]:
DATASET_PATH = Path('../dataset/Data')

train_paths = pd.read_csv(DATASET_PATH / "train_paths.csv")
valid_paths = pd.read_csv(DATASET_PATH / "valid_paths.csv")
test_paths = pd.read_csv(DATASET_PATH / "test_paths.csv")

## Functions

In [10]:
def preprocess_image(image):
    # Apply Gaussian Blur
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    # Apply Edge Detection (Canny)
    edges = cv2.Canny(blurred, 100, 200)
    return edges

def load_images(annotation):
    images = []
    heights = []
    widhts = []
    for path in annotation['path'].values:
        image = cv2.imread(path, flags=cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (516, 382))
        image = preprocess_image(image)
        images.append(image)
        heights.append(np.shape(image)[0])
        widhts.append(np.shape(image)[1])
    # print((np.min(heights) - np.max(heights)) // 2)
    # print((np.min(widhts) - np.max(widhts)) // 2)
    return np.asarray(images)

train_images = load_images(train_paths)
valid_images = load_images(valid_paths)
test_images = load_images(test_paths)

In [ ]:
def extract_combined_features(image, upper_limit=100000):
    # SIFT Feature Extraction
    sift = cv2.ORB_create()
    keypoints_sift, descriptors_sift = sift.detectAndCompute(image, None)

    # Akaze Feature Extraction
    akaze = cv2.AKAZE_create()
    keypoints_akaze, descriptors_akaze = akaze.detectAndCompute(image, None)

    # Handle None descriptors by replacing with zeros
    if descriptors_sift is None:
        descriptors_sift = np.zeros((1, 128))  # SIFT descriptors are 128-dimensional
    if descriptors_akaze is None:
        descriptors_akaze = np.zeros((1, 32))   # akaze descriptors are 32-dimensional

    # Flatten and concatenate features
    combined_features = np.concatenate(
        [descriptors_sift.flatten(), descriptors_akaze.flatten()]
    )

    # Adjust feature length to match upper_limit
    if len(combined_features) > upper_limit:
        # Truncate if features exceed the upper limit
        combined_features = combined_features[:upper_limit]
    elif len(combined_features) < upper_limit:
        # Pad with zeros if features are less than the upper limit
        padding_length = upper_limit - len(combined_features)
        combined_features = np.pad(combined_features, (0, padding_length), mode='constant')

    return np.asarray(combined_features)

combined_features = []
for image in train_images:
    feature = extract_combined_features(image)
    combined_features.append(feature)

In [24]:


def match_keypoints_between_images(image1, image2):
    # Preprocess images
    preprocessed1 = preprocess_image(image1)
    preprocessed2 = preprocess_image(image2)

    # Extract features
    combined_features1, keypoints1, _ = extract_combined_features(preprocessed1)
    combined_features2, keypoints2, _ = extract_combined_features(preprocessed2)

    # Brute Force Matcher
    sift = cv2.SIFT_create()
    _, descriptors1 = sift.detectAndCompute(preprocessed1, None)
    _, descriptors2 = sift.detectAndCompute(preprocessed2, None)
    matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

    # Match descriptors and apply Lowe's ratio test
    raw_matches = matcher.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = [m for m, n in raw_matches if m.distance < 0.75 * n.distance]

    return keypoints1, keypoints2, good_matches

def match_keypoints(desc1, desc2):
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(desc1, desc2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

In [ ]:
def train_classifier(features, labels):
    scaler = StandardScaler()
    encoder = LabelEncoder()
    labels = encoder.fit_transform(labels)
    # Split dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)

    # Train an SVM classifier
    model = XGBClassifier()
    model.fit(X_train, y_train)

    # Predict and evaluate
    predictions = model.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    return model

model = train_classifier(combined_features, train_paths['class'].values)